<a href="https://colab.research.google.com/github/tbloch1/fluxnet/blob/main/Themis_Goes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import glob
import requests
import multiprocessing as mp
import time
import warnings
import io
import multiprocessing as mp

from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec

try:
  import sunpy
except:
  !pip install sunpy
  clear_output()

try:
  import cdflib
except:
  !pip install cdflib
  import cdflib
  clear_output()

from sklearn.preprocessing import StandardScaler as SS
from sklearn.preprocessing import MinMaxScaler as MMS

plt.rcParams.update({'font.size': 8})

In [63]:
filepath = 'data/'
if not os.path.exists(filepath):
    os.makedirs(filepath)

In [3]:
def get_electron_data(spacecraft, year, month,mp=False):
    url = ('https://cdaweb.gsfc.nasa.gov/'+
           'pub/data/goes/goes{}/'.format(spacecraft)+
           'epead-electrons/e13ew_1min/'+
           '{}/goes{}_epead-science'.format(year,spacecraft)+
           '-electrons-e13ew_1min_'+
           '{}{:02d}01_v01.cdf'.format(year,month))

    data = requests.get(url)
    if data.status_code == 200:
        filename = filepath+'{}_{}_{:02d}'.format(spacecraft,year, month)
        with open(filename+'.cdf', 'wb') as f:
            f.write(data.content)
        goes = cdflib.CDF(filename+'.cdf')

        time = goes.varget('Epoch') # CDF epoch TT2000
        time = cdflib.epochs.CDFepoch.unixtime(time) # Unix Epoch
        en08 = goes.varget('E1W_COR_FLUX')
        en2 = goes.varget('E2W_COR_FLUX')
        en08err = goes.varget('E1W_COR_ERR')
        en2err = goes.varget('E2W_COR_ERR')
        en08q = goes.varget('E1W_DQF')
        en2q = goes.varget('E2W_DQF')

        goes_df = pd.DataFrame([time,en08,en2,en08err,
                                en2err,en08q,en2q],
                                index=['epoch_time',
                                       '800kevflux',
                                       '2mevflux',
                                       '800kevstd',
                                       '2mevstd',
                                       '800kevqual',
                                       '2mevqual']).T
        
        goes_df = goes_df.astype({'800kevflux':'float32',
                                    '2mevflux':'float32',
                                    '800kevstd':'float32',
                                    '2mevstd':'float32',
                                    '800kevqual':'float32',
                                    '2mevqual':'float32'})
        if not mp:
            return goes_df
        if mp:
            goes_df.to_parquet(filepath+
                               'goes{}_electron_{}_{}.parquet'.format(spacecraft,
                                                                      year,
                                                                      month))
    else:
        return False

In [4]:
def get_ephemeris_data(spacecraft,year,mp=False):
    url = ('https://cdaweb.gsfc.nasa.gov/pub/data/goes/'+
           'goes{}/orbit/{}/goes{}'.format(spacecraft,year,spacecraft)+
           '_ephemeris_ssc_{}0101_v01.cdf'.format(year))
    data = requests.get(url)
    
    filename = filepath+'{}_{}_01_ephemeris'.format(spacecraft,year)
    if data.status_code == 200:
        with open(filename+'.cdf', 'wb') as f:
            f.write(data.content)
        goes = cdflib.CDF(filename+'.cdf')

        time = goes.varget('Epoch') # CDF Epoch 0
        time = cdflib.epochs.CDFepoch.unixtime(time, to_np=True) # Unix Epoch
        xyzgsm = goes.varget('XYZ_GSM')
        xyzgse = goes.varget('XYZ_GSE')
        xyzgm = goes.varget('XYZ_GM')

        goes_df = pd.DataFrame(np.concatenate((time.reshape(-1,1),
                                            xyzgsm, xyzgse,xyzgm),axis=1),
                            columns=['epoch_time',
                                    'xgsm','ygsm','zgsm',
                                    'xgse','ygse','zgse',
                                    'xgm','ygm','zgm'])
        if not mp:
            return goes_df
        if mp:
            goes_df.to_parquet(filepath+
                               'goes{}_ephemeris_{}.parquet'.format(spacecraft,
                                                                    year))
    else:
        return False

In [5]:
inputs1 = [(spacecraft,year,month,True) for spacecraft in range(13,16)
           for year in range(2010,2021) for month in range(1,13)]

pool = mp.Pool(mp.cpu_count())
pool.starmap(get_electron_data,inputs1)
pool.close()

In [6]:
inputs2 = [(spacecraft,year,True) for spacecraft in range(13,16)
           for year in range(2010,2021)]
           
pool = mp.Pool(mp.cpu_count())
pool.starmap(get_ephemeris_data,inputs2)
pool.close()

In [36]:
for spacecraft in range(13,16):
    files = glob.glob(filepath+'goes{}_electron*.parquet'.format(spacecraft))
    elec_df = pd.concat([pd.read_parquet(i) for i in files])

    files = glob.glob(filepath+'goes{}_ephemeris*.parquet'.format(spacecraft))
    ephem_df = pd.concat([pd.read_parquet(i) for i in files])

    elec_df.index = [dt.datetime(1970,1,1)+dt.timedelta(seconds=i)
             for i in elec_df.epoch_time]
    ephem_df.index = [dt.datetime(1970,1,1)+dt.timedelta(seconds=i)
                for i in ephem_df.epoch_time]

    elec_df = elec_df.sort_index()
    ephem_df = ephem_df.sort_index().resample('1T').mean().interpolate(method='linear',
                                                                       limit=2)
    ephem_df = ephem_df.drop('epoch_time',axis=1)
    ephem_df['mlt'] = 24*(np.arctan2(ephem_df.ygsm,ephem_df.xgsm)+np.pi)/(2*np.pi)
    pd.concat([elec_df,ephem_df],axis=1,
              join='inner').to_parquet(filepath+
                                       'goes{}.parquet'.format(spacecraft))

In [64]:
# display(elec_df.head())
# display(ephem_df.head())

In [25]:
# df1 = elec_df.copy()
# df2 = ephem_df.copy()

In [45]:
# df = pd.read_parquet('goes13.parquet')

In [65]:
# plt.hist(df.xgm,bins=100)
# plt.hist(df.ygm,bins=100)
# plt.hist(df.zgm,bins=100)
# plt.show()